In [26]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from dataset_prep import create_total_dataframe
from data_preprocessing import preprocess_text

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 500000)

In [2]:
all_docx = ["data/2022_satellite.docx", "data/2021_satellite.docx"]
# all_docx = ["data/2020_satellite.docx"]

total_df = create_total_dataframe(all_docx)

/Users/darinaponomarova/Study/DiplomaMasters/dataset_prep.py:54: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  df['Authors'], df['Article_Name'] = df['Article_Name'].str.rsplit('.', 1).str


In [4]:
total_df['Processed_Text'] = total_df['Text'].apply(preprocess_text)

In [102]:
# total_df.head()

___

In [93]:
# Input text:
provided_text = "This ability to process massive datasets ensures that issues and dependencies that would otherwise go unnoticed are captured during modeling and improve the quality of the proposed mitigation plan."
provided_text = preprocess_text(provided_text)

In [97]:
# Vectorize:
vectorizer = TfidfVectorizer()
# vectorizer = TfidfVectorizer(ngram_range=(2,3))

corpus_tfidf = vectorizer.fit_transform(total_df['Processed_Text'])

In [98]:
provided_text_tfidf = vectorizer.transform([provided_text])
cosine_similarities = cosine_similarity(provided_text_tfidf, corpus_tfidf)

In [99]:
# Идентификация документов с высоким уровнем сходства
threshold = 0.1

for idx, similarity in enumerate(cosine_similarities[0]):
    if similarity > threshold:
        print(f"Document '{total_df.iloc[idx]['Article_Name']}' is similar with a score of {similarity}")

Document 'Possibilities of Project Management Using Artificial Intelligence' is similar with a score of 0.1618310281052039
Document 'Specifics of Project Quality Management' is similar with a score of 0.14302117734687064


In [103]:
total_df[total_df['Article_Name'] == 'Possibilities of Project Management Using Artificial Intelligence']

Article_Name Page  \
7  Possibilities of Project Management Using Artificial Intelligence   30   

                     Authors  \
7  Chyzhenko V., Kubiavka L.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            